# E2E AI Agent Testing

This notebook tests the AI Agent API endpoint `/api/chat` end-to-end.
It validates:
1. Response streaming
2. Planning phase
3. Tool execution
4. Reflection phase
5. Final output

In [ ]:
import requests
import json
import time
import uuid

BASE_URL = "http://localhost:3000/api/chat"

def stream_agent(message, thread_id=None, model="global.anthropic.claude-sonnet-4-5-20250929-v1:0", debug=False):
    if thread_id is None:
        thread_id = str(uuid.uuid4())
    
    payload = {
        "messages": [
            {"role": "user", "content": message}
        ],
        "threadId": thread_id,
        "model": model,
        "autoApprove": True
    }
    
    print(f"--- Starting Request [Thread: {thread_id}] ---")
    print(f"User: {message}\n")
    
    try:
        with requests.post(BASE_URL, json=payload, stream=True) as response:
            response.raise_for_status()
            
            for line in response.iter_lines():
                if line:
                    decoded_line = line.decode('utf-8')
                    if debug:
                        print(f"[DEBUG] {decoded_line}")
                    
                    # Vercel AI SDK Data Stream Protocol parsing
                    # 0:text - Text parts
                    # 2:json - Data parts (tool events, etc)
                    # 8:json - Data parts (alternative)
                    # e:json - Error parts
                    
                    if decoded_line.startswith('0:'):
                        # Text delta
                        content = decoded_line[2:]
                        # Deserialize JSON string if it's quoted (simple checks)
                        if content.startswith('"') and content.endswith('"'):
                             try:
                                 content = json.loads(content)
                             except:
                                 pass
                        print(content, end="", flush=True)
                        
                    elif decoded_line.startswith('2:'):
                        # Data part - usually contains our custom tool protocol
                        content = decoded_line[2:]
                        try:
                            data = json.loads(content)
                            # Handle array of data items
                            if isinstance(data, list):
                                items = data
                            else:
                                items = [data]
                                
                            for item in items:
                                if isinstance(item, dict):
                                    msg_type = item.get('type')
                                    
                                    if msg_type == 'reasoning-start':
                                        print("\n\n[REASONING STARTED]", flush=True)
                                    elif msg_type == 'reasoning-delta':
                                        print(item.get('delta', ''), end="", flush=True)
                                    elif msg_type == 'reasoning-end':
                                        print("\n[REASONING ENDED]\n", flush=True)
                                        
                                    elif msg_type == 'tool-input-start':
                                        tool_name = item.get('toolName', 'unknown')
                                        print(f"\n\n🔧 [TOOL CALL: {tool_name}]", flush=True)
                                    elif msg_type == 'tool-input-available':
                                        args = item.get('input', {})
                                        print(f"   Args: {json.dumps(args, indent=2)}", flush=True)
                                    elif msg_type == 'tool-output-available':
                                        output = str(item.get('output', ''))
                                        print(f"   Output: {output[:300]}..." if len(output) > 300 else f"   Output: {output}", flush=True)
                                        print("   [TOOL COMPLETED]\n", flush=True)
                                        
                        except json.JSONDecodeError:
                            pass
                            
                    elif decoded_line.startswith('e:'):
                        # Error
                        print(f"\n[SERVER ERROR] {decoded_line[2:]}")
                        
    except Exception as e:
        print(f"\nError: {e}")
    
    return thread_id


### Test Case 1: Simple Connectivity
Check if the server is up and responsive.

In [ ]:
stream_agent("Hello, are you online?")

### Test Case 2: Planning & Execution
Ask the agent to perform a task that requires tools (listing the directory).

In [ ]:
tid = stream_agent("List the contents of the current directory and tell me what files are there.")

### Test Case 3: Complex Multi-step (Reflection)
Ask a question that might require some thinking or multiple steps.

In [ ]:
stream_agent("Create a plan to research 'LangGraph' and then summarize its key features.")